In [ ]:
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))


reading dataset

# using back-translation technique for data augmentation

In [ ]:
from deep_translator import GoogleTranslator

farsi_to_english_translator = GoogleTranslator(source='fa',target='en')
english_to_farsi_translator = GoogleTranslator(source='en',target='fa')

augmented_data = []
data = []

with open('/content/drive/MyDrive/temp_v3.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    data.append(result)

    english_question = farsi_to_english_translator.translate(result['question'])
    reformed_question = english_to_farsi_translator.translate(english_question)

    new_data = {}
    new_data['question'] = reformed_question
    new_data['context'] = result['context']
    new_data['pq_id'] = result['pq_id']
    new_data['answers'] = result['answers']

    augmented_data.append(new_data)

final_data = data + augmented_data


In [ ]:
output_path = '/content/drive/MyDrive/backtranslated.jsonl'
dump_jsonl(augmented_data, output_path)

Wrote 2088 records to /content/drive/MyDrive/backtranslated.jsonl


In [ ]:
output_path = '/content/drive/MyDrive/AugmentedDataset.jsonl'
dump_jsonl(final_data, output_path)

Wrote 4176 records to /content/drive/MyDrive/AugmentedDataset.jsonl


In [ ]:
for each in augmented_data:
  print(each['question'])

In [ ]:
with open('/content/drive/MyDrive/aug_train.json', 'w') as f:
   json.dump(final_data[:3340], f)

with open('/content/drive/MyDrive/aug_dev.json', 'w') as f:
   json.dump(final_data[3340:], f)